In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [10]:
# Load and preprocess the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data (important for deep learning models)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [11]:
datagen = ImageDataGenerator(
    rotation_range=30,  # Increased rotation range
    width_shift_range=0.2,  # Increased width shift range
    height_shift_range=0.2,  # Increased height shift range
    shear_range=0.2,  # Increased shear range
    zoom_range=0.2,  # Increased zoom range
    horizontal_flip=True,  # Keep flipping
    fill_mode='nearest'  # Strategy for filling in missing pixels after transformation
)

# Fit the generator on the training data
datagen.fit(x_train)

In [12]:
def build_wrn_model():
    num_blocks = 3  # Number of blocks in the model
    block_sizes = [16, 32, 64]  # Number of filters in each block
    growth_rate = 10  # Wide ResNet scaling factor

    inputs = layers.Input(shape=(32, 32, 3))

    # Initial Conv Layer with padding to prevent rapid dimension reduction
    x = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)

    # Add Residual Blocks with adjusted MaxPooling layers
    for i in range(num_blocks):
        for _ in range(block_sizes[i] // 16):  # The number of residual blocks in each layer
            x = layers.Conv2D(block_sizes[i], (3, 3), padding='same', activation='relu')(x)
            x = layers.BatchNormalization()(x)
            x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)

    # Global Average Pooling + Dense layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(10, activation='softmax')(x)

    model = Model(inputs, x)
    return model

In [13]:
# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

In [14]:
# Compile the updated model with higher learning rate
model = build_wrn_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train with updated architecture
model.fit(datagen.flow(x_train, y_train, batch_size=64),
          epochs=150,  # Increased epochs
          validation_data=(x_test, y_test),
          callbacks=[lr_scheduler])

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 50ms/step - accuracy: 0.3089 - loss: 1.9252 - val_accuracy: 0.4647 - val_loss: 1.5236 - learning_rate: 0.0010
Epoch 2/150
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 43ms/step - accuracy: 0.4617 - loss: 1.4798 - val_accuracy: 0.4757 - val_loss: 1.4280 - learning_rate: 0.0010
Epoch 3/150
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.5178 - loss: 1.3369 - val_accuracy: 0.5046 - val_loss: 1.3889 - learning_rate: 0.0010
Epoch 4/150
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.5500 - loss: 1.2585 - val_accuracy: 0.5519 - val_loss: 1.3662 - learning_rate: 0.0010
Epoch 5/150
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.5748 - loss: 1.1813 - val_accuracy: 0.6188 - val_loss: 1.0627 - learning_rate: 0.0010
Epoch 6/150
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.5998 - loss: 1.1319 - val_accuracy: 0.4646 - val_loss: 1.9389 - learning_rate: 0.0010
Epoch 7/150
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.6127 - loss: 